### 직방의 원룸 매물정보 수집
- 절차
    - 동이름 > 위도,경도 수집
    - 위도,경도 > geohash 변환
    - geohash(영역) > 매물 아이디
    - 매물 아이디 > 매물 정보

In [6]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=0c698f6462f6e1b2a0598c5906aa38a54823a01e8be92ddc3b6abd6a035fe414
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [7]:
import requests
import pandas as pd
import geohash2

In [1]:
# 1. 동이름 > 위도, 경도

In [33]:
#search?

In [13]:
addr = '서울시 강남구'
url =  f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"],data["lng"]
lat, lng

(37.5174064636231, 127.047309875488)

### url 디코딩 : https://meyerweb.com/eric/tools/dencoder/

In [8]:
# 개발자 도구 : ctrl+shift+I

In [2]:
# 2. 위도,경도 > geohash

In [27]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat,lng,precision=5)
geohash

'wydm7'

### 설명 보고 싶으면 shift + tab

In [3]:
# 3. geohash > 매물 아이디

In [15]:
# 공백문자에 \ 쓰면 에러
# 내용이 길때 한줄 띄우려면 \

In [32]:
# items?deposit

In [29]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [30]:
items = response.json()["items"]
len(items), items[:2]

(707,
 [{'lat': 37.48984284564423, 'lng': 127.08260954244908, 'item_id': 32405381},
  {'lat': 37.49005074959576, 'lng': 127.08382572848865, 'item_id': 32635864}])

In [34]:
ids = [item["item_id"] for item in items]
ids[:5]

[32405381, 32635864, 32666863, 32730696, 32875403]

In [4]:
# 4. 매물 아이디 > 매물 정보

In [35]:
# list
# post방식일땐 param 써야됨 param 정보는 payload에서 확인 가능

In [37]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain":"zigbang",
    "withCoalition":"true",
    "item_ids":ids[:900]
}
response = requests.post(url,params)
response

<Response [200]>

In [38]:
items = response.json()['items']
columns = ['item_id',"sales_type","deposit",'rent','size_m2','address1','manage_cost']
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
705,32780815,전세,33000,0,62.84,서울시 광진구 자양동,2
706,32872378,전세,32000,0,85.95,서울시 광진구 자양동,2


In [41]:
# max row, max column 설정
pd.options.display.max_columns, pd.options.display.max_rows

(50, 60)

In [39]:
pd.options.display.max_columns = 50

In [40]:
pd.DataFrame(items).tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
705,None,32780815,https://ic.zigbang.com/ic/items/32780815/1.jpg,전세,전세,33000,0,62.84,"{'m2': 62.84, 'p': '19'}","{'m2': 62.81, 'p': '19'}",None,None,4,4,5,"●리모델링●공간,채광 좋은●3룸",None,05,광진구 자양동,"{'lat': 37.52954654800517, 'lng': 127.08253347...",False,True,빌라,[],서울시 광진구 자양동,None,None,2,2022-07-27T14:03:31+09:00,False
706,None,32872378,https://ic.zigbang.com/ic/items/32872378/1.jpg,전세,전세,32000,0,85.95,"{'m2': 85.95, 'p': '26'}","{'m2': 82.64, 'p': '25'}",None,None,4,4,5,아파트36평형 크기의 3룸.방3개 모두큼.주차가능.입주하시,None,05,광진구 자양동,"{'lat': 37.52955396805742, 'lng': 127.08257926...",False,True,빌라,[],서울시 광진구 자양동,None,None,2,2022-08-03T18:52:25+09:00,True


In [49]:
# 함수 만들기
def oneroom(addr):
    
    url =  f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"],data["lng"]
    
    geohash = geohash2.encode(lat,lng,precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}\
&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain":"zigbang",
        "withCoalition":"true",
        "item_ids":ids[:900]
    }
    response = requests.post(url,params)
    
    items = response.json()['items']
    columns = ['item_id',"sales_type","deposit",'rent','size_m2','address1','manage_cost']
    return pd.DataFrame(items)[columns]

In [50]:
addr ='마포구 합정동'
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
745,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [51]:
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [52]:
# 모듈 파일 만들기:.py

In [54]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

def oneroom(addr):
    
    url =  f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"],data["lng"]
    
    geohash = geohash2.encode(lat,lng,precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}\
&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain":"zigbang",
        "withCoalition":"true",
        "item_ids":ids[:900]
    }
    response = requests.post(url,params)
    
    items = response.json()['items']
    columns = ['item_id',"sales_type","deposit",'rent','size_m2','address1','manage_cost']
    return pd.DataFrame(items)[columns]

Writing zigbang.py


In [58]:
# writefile 위에 주석 있으면 안됨

In [56]:
import zigbang as zb

In [57]:
df = zb.oneroom('망원동')
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
745,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [60]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [61]:
data = 1

In [62]:
%whos

Variable   Type    Data/Info
----------------------------
data       int     1
